<center style="font-size: 2em; font-weight: bold;"> Retornos a la Educacion </center>
<center style="font-size: 2em; font-weight: bold;"> Variables Instrumentales  </center>
<left style="font-size: 4em; font-weight: bold;"> Autor: Edinson Tolentino </left>

# Librerias

In [1]:
import pandas
import numpy
import os
import sys
import numpy

import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

import sklearn
import sklearn.compose
#import category_encoders
import sklearn.impute

from plotnine import *
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

#import py_hep_functions
import seaborn
import matplotlib.pyplot as plt

# Metodo de Variable instrumentales (recurso)
# https://bashtage.github.io/linearmodels/iv/examples/basic-examples.html

In [2]:

color = ["#3a5e8cFF", "#10a53dFF", "#541352FF", "#ffcf20FF", "#2f9aa0FF"]

def seq(start: float, stop: float, by: float, round_n=3) -> list:
    """
    Custom function, used for setting the breaks of plotnine scales.

       Parameters
    ----------
    start : float
        Start of the breaks.
    stop : float
        End of the breaks.
    by : float
        Steps between breaks.
    round_n: int, default=3
        Decimals to round floats in output.
    """
    epsilon = numpy.finfo("float").eps

    return [
        round(x, round_n) for x in list(numpy.arange(start, stop + (by - epsilon), by))
    ]
    
def normalize_frame(frame):
    """Normalize the data frame to make it performant and compatible with
    downstream libraries such as Scikit-Learn and CatBoost.
    
    In particular the following operations are performed:
    - Categorical levels that are not strings are converted into strings
    - Categorical missing values are converted into a distinct "N/A" level
    - Object-typed columns are converted into categorical columns
    """
    for col, series in frame.items():
        dtype = series.dtype
        if pandas.api.types.is_categorical_dtype(dtype):
            recode = False
            if series.hasnans:
                recode = True
            else:
                for value in series.cat.categories:
                    if not isinstance(value, str):
                        recode = True
                        break
            if recode:
                frame[col] = series.astype("str").fillna("N/A").astype(
                    "category")
        elif pandas.api.types.is_object_dtype(dtype):
            frame[col] = series.astype("str").fillna("N/A").astype("category")

    return frame

base_transformer = sklearn.pipeline.Pipeline([
    ("recode", sklearn.compose.ColumnTransformer(transformers=[
        ("category",
             #sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore",
             sklearn.preprocessing.OneHotEncoder(drop="first",handle_unknown="ignore",                                                 
                min_frequency=0.01, max_categories=30, sparse_output= False),
             sklearn.compose.make_column_selector(dtype_exclude=numpy.number)
        ),
        ("number",
            sklearn.impute.SimpleImputer(strategy="mean"),
            sklearn.compose.make_column_selector(dtype_include=numpy.number)
        ),  
    ], n_jobs=-1)),
    #("rescale", sklearn.preprocessing.StandardScaler()),
])

# Importar la datasets

In [3]:
output           = "C:/Users/et396/Dropbox/Docencia/Educate/Econometria/S5/Data"
os.chdir(output)

# Carga de data
base = pandas.read_stata("BD5.dta")

# Manejo de Data

In [4]:
# Filtro de variables
response = ['logwage']
inputs   = ['female', 'educ', 'test11r', 'test11m', 'mumso', 'dadso']
data_analysis = base[response + inputs]

# Normalise informacion : number, category columns
data_fit = normalize_frame(data_analysis)
data_fit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   logwage  1000 non-null   float32
 1   female   1000 non-null   float32
 2   educ     1000 non-null   float32
 3   test11r  1000 non-null   float32
 4   test11m  1000 non-null   float32
 5   mumso    1000 non-null   float32
 6   dadso    1000 non-null   float32
dtypes: float32(7)
memory usage: 27.5 KB


C:\Users\et396\AppData\Local\Temp\ipykernel_30724\2308428612.py:35: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead


# Pregunta 1

## Modelo MCO

In [5]:
# Independed variabkes
response = data_fit[['logwage']]

# Depended variables
predictors_cols1 = ['female', 'educ']
predictors_cols2 = ['female', 'educ', 'test11r', 'test11m', 'mumso', 'dadso']


In [6]:
base1       = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m1 = sm.OLS( response, inputs).fit(cov_type="HC1")
print(m1.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     145.8
Date:                Sun, 31 Mar 2024   Prob (F-statistic):           2.93e-56
Time:                        16:14:16   Log-Likelihood:                -611.05
No. Observations:                1000   AIC:                             1228.
Df Residuals:                     997   BIC:                             1243.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              1.6949      0.032     53.

In [7]:
stargazer = Stargazer([m1])
stargazer.custom_columns(["log wage"], [1])
stargazer.covariate_order(["number__educ", 'number__female'])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos individuos", ["No"])
stargazer.add_line("Efectos fijos Year", ["No"])
stargazer

# Pregunta 3

## Modelo forma reducida

In [8]:
# Independed variabkes
response = data_fit[['educ']]

# Depended variables
predictors_cols1 = ['female', 'test11r', 'test11m', 'mumso', 'dadso']

In [9]:
base1       = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m2 = sm.OLS( response, inputs).fit(cov_type="HC1")

In [10]:
inputs.columns

Index(['const', 'number__female', 'number__test11r', 'number__test11m',
       'number__mumso', 'number__dadso'],
      dtype='object')

In [11]:
stargazer = Stargazer([m2])
stargazer.custom_columns(["Educ"], [1])
stargazer.covariate_order(["number__female", 'number__test11r', 'number__test11m', 'number__mumso', 'number__dadso'])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos individuos", ["No"])
stargazer.add_line("Efectos fijos Year", ["No"])
stargazer

## Modelo IV manualmente

In [12]:
data_fit['educhat'] = m2.predict(inputs)

# Variable independiente
response = data_fit[['logwage']]

# Depended variables
predictors_cols1 = ['female', 'educhat']

# Modelo IV manualmente
base1       = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m3 = sm.OLS( response, inputs).fit(cov_type="HC1")

In [13]:
print(m3.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.199
Model:                            OLS   Adj. R-squared:                  0.197
Method:                 Least Squares   F-statistic:                     132.0
Date:                Sun, 31 Mar 2024   Prob (F-statistic):           1.36e-51
Time:                        16:45:06   Log-Likelihood:                -626.66
No. Observations:                1000   AIC:                             1259.
Df Residuals:                     997   BIC:                             1274.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.5714      0.038     

In [14]:
stargazer = Stargazer([m3])
stargazer.custom_columns(["log wage"], [1])
stargazer.covariate_order(["number__educhat", 'number__female'])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos individuos", ["No"])
stargazer.add_line("Efectos fijos Year", ["No"])
stargazer

## Presentacion MCO vs IV

In [15]:
stargazer = Stargazer([m1, m3])
stargazer.custom_columns(["MCO" , "IV"], [1, 1])
stargazer.covariate_order(["number__educhat", "number__educ", 'number__female'])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos individuos", ["No","No"])
stargazer.add_line("Efectos fijos Year", ["No","No"])
stargazer

# Pregunta 4-5

In [16]:
from linearmodels.iv import IV2SLS

# Variable independiente
response = data_fit[['logwage']]

# Depended variables
predictors_cols1 = ['female']
instrument_cols1 = ['test11r', 'test11m', 'mumso', 'dadso']

# Modelo IV manualmente
base1       = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m4 = IV2SLS( response, inputs, data_fit.educ ,data_fit[instrument_cols1]).fit()


In [17]:
print(m4)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                logwage   R-squared:                      0.1993
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1976
No. Observations:                1000   F-statistic:                    270.86
Date:                Sun, Mar 31 2024   P-value (F-stat)                0.0000
Time:                        17:17:20   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const              1.5714     0.0378     41.554     0.0000      1.4973      1.6455
number__female    -0.2829     0.0288

In [18]:
print(m4.first_stage)

    First Stage Estimation Results    
                                  educ
--------------------------------------
R-squared                       0.3856
Partial R-squared               0.3821
Shea's R-squared                0.3821
Partial F-statistic             677.61
P-value (Partial F-stat)        0.0000
Partial F-stat Distn           chi2(4)
========================== ===========
const                           0.3042
                              (1.6721)
number__female                 -0.0562
                             (-0.4839)
test11r                         0.0321
                              (6.3779)
test11m                         0.0307
                              (9.2446)
mumso                           0.5667
                              (3.7402)
dadso                           0.5393
                              (3.3066)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


## Test de Sargan

In [19]:
data_fit['rmu'] = m4.resids


In [22]:
# Variable independiente
response = data_fit[['rmu']]

# Depended variables
predictors_cols1 = ['female', 'test11r', 'test11m', 'mumso', 'dadso']

# Regression de los errores
base1       = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m5 = sm.OLS( response, inputs).fit(cov_type="HC1")

In [23]:
print(m5.summary())

                            OLS Regression Results                            
Dep. Variable:                    rmu   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.765
Date:                Sun, 31 Mar 2024   Prob (F-statistic):              0.117
Time:                        17:22:24   Log-Likelihood:                -621.97
No. Observations:                1000   AIC:                             1256.
Df Residuals:                     994   BIC:                             1285.
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0277      0.042     

In [24]:
stargazer = Stargazer([m5])
stargazer.custom_columns(["mu"], [1])
stargazer.covariate_order(["number__female", 'number__test11r', 'number__test11m', 'number__mumso', 'number__dadso'])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos individuos", ["No"])
stargazer.add_line("Efectos fijos Year", ["No"])
stargazer

In [25]:
m4.wooldridge_overid

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 8.6666
P-value: 0.0341
Distributed: chi2(3)
WaldTestStatistic, id: 0x21a88ed83e0

# Pregunta 6

In [26]:
data_fit['rerror'] = m2.resid

In [27]:
# Variable independiente
response = data_fit[['logwage']]

# Depended variables
predictors_cols1 = ['female','educ', 'rerror']

# Modelo IV manualmente
base1       = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m6 = sm.OLS( response, inputs).fit(cov_type="HC1")

In [28]:
stargazer = Stargazer([m6])
stargazer.custom_columns(["log wage"], [1])
stargazer.covariate_order(["number__educ", 'number__female','number__rerror'])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos individuos", ["No"])
stargazer.add_line("Efectos fijos Year", ["No"])
stargazer

In [29]:
# Test de exogenidad
m4.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 18.8200
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x21a88f209b0

In [30]:
from linearmodels.iv import compare

# Variable independiente
response = data_fit[['logwage']]

# Depended variables
predictors_cols1 = ['female']
instrument_cols1 = ['test11r', 'test11m', 'mumso', 'dadso']

# Modelo IV manualmente
base1       = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

# Modificamos el datafit agregando constantes
predictors_cols2 = ['female', 'educ']
base2       = data_fit[predictors_cols2]
predictors  = data_fit[predictors_cols2]
transformer = sklearn.clone(base_transformer)
inputs2      = transformer.fit_transform(predictors)
inputs2      = pandas.DataFrame(inputs2, index=base2.index,
                               columns=transformer.get_feature_names_out())
inputs2      = sm.add_constant(inputs2, has_constant="raise")


m7 = IV2SLS( response, inputs2 ,None,  None).fit()
m8 = IV2SLS( response, inputs, data_fit.educ ,data_fit[instrument_cols1]).fit()


In [31]:
print(compare({"OLS": m7, "2SLS": m8}))

                Model Comparison                
                               OLS          2SLS
------------------------------------------------
Dep. Variable              logwage       logwage
Estimator                      OLS       IV-2SLS
No. Observations              1000          1000
Cov. Est.                   robust        robust
R-squared                   0.2237        0.1993
Adj. R-squared              0.2221        0.1976
F-statistic                 292.41        270.86
P-value (F-stat)            0.0000        0.0000
==================     ===========   ===========
const                       1.6949        1.5714
                          (53.352)      (41.554)
number__female             -0.2949       -0.2829
                         (-10.341)     (-9.8184)
number__educ                0.0761              
                          (11.739)              
educ                                      0.1101
                                        (12.156)
====================